In [1]:
# db_config.py
# Centralized database connection details.
# In a real-world scenario, use environment variables or a secrets manager.

DB_CONFIG = {
    'host': 'localhost',
    'user': 'py_user',
    'password': 'password',
    'database': 'health_analytics'
}
```python
# healthcare/__init__.py
# Makes the 'healthcare' directory a Python package.
```python
# healthcare/database.py
# This module encapsulates all database interactions.

import mysql.connector
from mysql.connector import Error

class Database:
    """
    A class to manage connections and transactions with the MySQL database.
    This class handles the low-level details of connecting, executing queries,
    and fetching results, providing a clean interface for other parts of the application.
    """
    def __init__(self, db_config):
        """
        Initializes the Database object.

        Args:
            db_config (dict): A dictionary containing database connection parameters.
        """
        self.config = db_config
        self.connection = None

    def connect(self):
        """
        Establishes a connection to the database.
        It's designed to be called before any database operation.
        """
        try:
            self.connection = mysql.connector.connect(**self.config)
            if self.connection.is_connected():
                print("Successfully connected to the database.")
        except Error as e:
            print(f"Error while connecting to MySQL: {e}")
            self.connection = None # Ensure connection is None on failure

    def disconnect(self):
        """
        Closes the database connection if it is open.
        """
        if self.connection and self.connection.is_connected():
            self.connection.close()
            print("Database connection closed.")

    def execute_query(self, query, params=None, multi=False):
        """
        Executes a given SQL query (e.g., INSERT, UPDATE, DELETE).

        Args:
            query (str): The SQL query to execute.
            params (tuple, optional): A tuple of parameters to pass to the query.
                                      This helps prevent SQL injection. Defaults to None.
            multi (bool): Set to True if executing multiple statements from a script.

        Returns:
            int: The last inserted row ID if applicable, otherwise None.
        """
        if not self.connection or not self.connection.is_connected():
            print("Cannot execute query: Not connected to the database.")
            return None

        cursor = self.connection.cursor()
        last_row_id = None
        try:
            if multi:
                # For executing scripts with multiple statements
                for result in cursor.execute(query, multi=True):
                    pass
            else:
                cursor.execute(query, params)

            self.connection.commit()
            last_row_id = cursor.lastrowid
            print("Query executed successfully.")
        except Error as e:
            print(f"Error executing query: {e}")
            self.connection.rollback()
        finally:
            cursor.close()
        return last_row_id

    def fetch_all(self, query, params=None):
        """
        Fetches all rows from a SELECT query.

        Args:
            query (str): The SELECT query to execute.
            params (tuple, optional): Parameters to bind to the query.

        Returns:
            list: A list of tuples representing the rows, or an empty list on failure.
        """
        if not self.connection or not self.connection.is_connected():
            print("Cannot fetch data: Not connected to the database.")
            return []

        cursor = self.connection.cursor(dictionary=True)
        results = []
        try:
            cursor.execute(query, params)
            results = cursor.fetchall()
        except Error as e:
            print(f"Error fetching data: {e}")
        finally:
            cursor.close()
        return results
```python
# healthcare/patient.py
# Data model for a Patient.

from datetime import date

class Patient:
    """
    Represents a patient record. This class acts as a data structure
    to hold patient information, making it easier to pass patient data
    between different parts of the application.
    """
    def __init__(self, patient_id, first_name, last_name, dob, gender):
        self.patient_id = patient_id
        self.first_name = first_name
        self.last_name = last_name
        self.dob = dob
        self.gender = gender

    def calculate_age(self):
        """Calculates the current age of the patient."""
        today = date.today()
        birth_date = self.dob
        age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
        return age

    def __str__(self):
        return f"Patient ID: {self.patient_id}, Name: {self.first_name} {self.last_name}, Age: {self.calculate_age()}"
```python
# healthcare/analysis.py
# Contains the core logic for performing healthcare analytics.

from .patient import Patient

class HealthAnalytics:
    """
    Provides methods to perform data analysis on patient records.
    This class depends on a Database object to retrieve data, separating
    the analysis logic from the data access logic.
    """
    def __init__(self, database):
        """
        Initializes the HealthAnalytics object.

        Args:
            database (Database): An instance of the Database class.
        """
        self.db = database

    def get_patient_by_last_name(self, last_name):
        """
        Retrieves patient records by last name.

        Args:
            last_name (str): The last name to search for.

        Returns:
            list: A list of Patient objects.
        """
        query = "SELECT * FROM patients WHERE last_name = %s"
        patients_data = self.db.fetch_all(query, (last_name,))

        patient_list = []
        for p_data in patients_data:
            patient = Patient(
                patient_id=p_data['patient_id'],
                first_name=p_data['first_name'],
                last_name=p_data['last_name'],
                dob=p_data['dob'],
                gender=p_data['gender']
            )
            patient_list.append(patient)
        return patient_list

    def get_total_patient_count(self):
        """
        Calculates the total number of patients in the database.

        Returns:
            int: The total count of patients.
        """
        query = "SELECT COUNT(*) as total FROM patients"
        result = self.db.fetch_all(query)
        return result[0]['total'] if result else 0

    def get_diagnoses_summary(self):
        """
        Generates a summary of diagnoses counts.

        Returns:
            list: A list of dictionaries with diagnosis and count.
        """
        query = """
            SELECT diagnosis, COUNT(*) as count
            FROM diagnoses
            GROUP BY diagnosis
            ORDER BY count DESC
            LIMIT 10;
        """
        return self.db.fetch_all(query)
```python
# data_importer.py
# Script to read patient data from a CSV file and import it into the database.

import csv
from db_config import DB_CONFIG
from healthcare.database import Database

def import_data(csv_file_path):
    """
    Reads patient data from a CSV and inserts it into the database.
    For simplicity, this example only populates the 'patients' table.
    A more complete implementation would populate all related tables.
    """
    db = Database(DB_CONFIG)
    db.connect()

    if not db.connection:
        print("Halting import due to database connection failure.")
        return

    insert_query = """
    INSERT INTO patients (first_name, last_name, dob, gender)
    VALUES (%s, %s, %s, %s)
    """

    try:
        with open(csv_file_path, mode='r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                params = (row['first_name'], row['last_name'], row['dob'], row['gender'])
                db.execute_query(insert_query, params)
        print(f"Successfully imported data from {csv_file_path}")
    except FileNotFoundError:
        print(f"Error: The file {csv_file_path} was not found.")
    except Exception as e:
        print(f"An error occurred during data import: {e}")
    finally:
        db.disconnect()

if __name__ == "__main__":
    # Assuming 100k records can be too slow for a live demo,
    # we'll use a smaller sample file name.
    # The generation script can create a larger file if needed.
    import_data('patients_sample.csv')
```python
# main_analysis.py
# The main entry point for the healthcare analytics application.

from db_config import DB_CONFIG
from healthcare.database import Database
from healthcare.analysis import HealthAnalytics

def main():
    """
    Main function to run the analytics engine.
    """
    # Initialize and connect to the database
    db = Database(DB_CONFIG)
    db.connect()

    if not db.connection:
        print("Exiting application due to database connection failure.")
        return

    # Create an instance of the analytics engine
    analytics = HealthAnalytics(db)

    # --- Perform Analysis ---

    # 1. Get total number of patients
    total_patients = analytics.get_total_patient_count()
    print(f"\n--- Analytics Report ---")
    print(f"Total number of patients in the database: {total_patients}")

    # 2. Find patients by last name
    print("\nSearching for patients with last name 'Smith'...")
    smith_patients = analytics.get_patient_by_last_name('Smith')
    if smith_patients:
        print(f"Found {len(smith_patients)} patients with the last name Smith:")
        for patient in smith_patients:
            print(f"  - {patient}")
    else:
        print("No patients with the last name Smith found.")

    # 3. Get a summary of the most common diagnoses
    print("\nTop 10 Most Common Diagnoses:")
    diagnoses_summary = analytics.get_diagnoses_summary()
    if diagnoses_summary:
        for item in diagnoses_summary:
            print(f"  - Diagnosis: {item['diagnosis']}, Count: {item['count']}")
    else:
        print("No diagnosis data found.")

    # Disconnect from the database
    db.disconnect()

if __name__ == "__main__":
    main()
```python
# requirements.txt
mysql-connector-python
```python
# generate_fake_data.py
# (Optional) Script to generate a large synthetic dataset.

import csv
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker()

def create_patient_data(record_count):
    """Generates a list of synthetic patient records."""
    patients = []
    for _ in range(record_count):
        patients.append({
            'first_name': fake.first_name(),
            'last_name': fake.last_name(),
            'dob': fake.date_of_birth(minimum_age=1, maximum_age=90).strftime('%Y-%m-%d'),
            'gender': random.choice(['Male', 'Female', 'Other'])
        })
    # Add a few common names for searching
    for _ in range(5):
         patients.append({
            'first_name': fake.first_name(),
            'last_name': 'Smith',
            'dob': fake.date_of_birth(minimum_age=1, maximum_age=90).strftime('%Y-%m-%d'),
            'gender': random.choice(['Male', 'Female'])
        })
    return patients

def write_csv(data, filename, fieldnames):
    """Writes data to a CSV file."""
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
    print(f"Generated {filename} with {len(data)} records.")

if __name__ == "__main__":
    # Generate a smaller sample for quick import
    patient_records_sample = create_patient_data(1000)
    patient_fields = ['first_name', 'last_name', 'dob', 'gender']
    write_csv(patient_records_sample, 'patients_sample.csv', patient_fields)

    # To generate the full 100k dataset (can be slow)
    # print("Generating 100,000 patient records... this might take a while.")
    # patient_records_large = create_patient_data(100000)
    # write_csv(patient_records_large, 'patients_100k.csv', patient_fields)

SyntaxError: invalid syntax (ipython-input-1-3984579156.py, line 11)